#### The DIO files
The DIO files are digital input-output files from the digital input-output board. The inputs to the DIO board are infrared sensors that detect motion near the reward well and trigger automatic delivery of reward. The DIO files are formatted as {*animal*}DIO{*day*}.mat. This notebook is an attempt to figure out how the data is formatted in the DIO files.

##### DIO .mat Variables
Each Matlab file has three Matlab-cells: *DIO*, *rawdio*, and *diopulses*. For the *DIO* and *rawdio* Matlab-cells, the data structure is nested in Matlab-cells {*day*}{*epoch*}{*pins*}. For the *diopulses* Matlab-cell, the structure is nested as {*pins*}. Pins are digital pins that can be configured as an input or an output. Inactive pins are empty cells. Active pins contain Matlab-structures.



In [727]:
%matplotlib inline 
import os
import collections
import scipy.io
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as sns

# Get an example DIO file assuming the data is in the Raw Data folder
Animal = collections.namedtuple('Animal', {'directory', 'short_name'})
HPa = Animal(directory='HPa_direct', short_name='HPa')
day = 8

def get_data_filename(animal, day, file_type):
    '''
    Returns the Matlab file name assuming it is in the Raw Data directory.
    File type is a string that refers to the various data structure names (DIO, tasks, linpos)
    Animal is a named tuple. Day is an integer giving the recording session day.
    '''
    data_dir = '{working_dir}/Raw-Data/'.format(working_dir=os.getcwd())
    return '{data_dir}/{animal.directory}/{animal.short_name}{file_type}{day:02d}.mat'.format(
        data_dir=data_dir,
        animal=animal,
        file_type=file_type,
        day=day)   

dio_file = get_data_filename(HPa, day, 'DIO')

# Load .mat file
print('DIO file: {file}'.format(file=dio_file))
DIO = scipy.io.loadmat(dio_file)

# Make sure all the Matlab-cells exist and are the right size
print('DIO size: {size}'.format(size=DIO['DIO'].shape))
assert(DIO['DIO'].shape[1] == day)

print('rawdio size: {size}'.format(size=DIO['rawdio'].shape))
assert(DIO['rawdio'].shape[1] == day)

print('diopulses size: {size}'.format(size=DIO['diopulses'].shape))

DIO file: /Users/edeno/Documents/GitHub/Jadhav-2016-Data-Analysis/Raw-Data//HPa_direct/HPaDIO08.mat
DIO size: (1, 8)
rawdio size: (1, 8)
diopulses size: (1, 24)


Define some accessor functions to get the positional data (pos) and the DIO data for a particular animal, day, epoch and environment

In [737]:
def get_epoch(animal, days, epoch_type='', environment=''):
    '''
    Returns a list of two-element tuples (day, epoch index) that
    can access the data structure within each Matlab file. Epoch type
    is the task for that epoch (sleep, run, etc.) and environment is
    the type of maze the animal is in (if any). If no epoch type or
    environment is given, returns all epoch types and environments
    '''
    if isinstance(days, int):
        days = [days]
    epoch_index = list()
    for day in days:
        try:
            task_file = scipy.io.loadmat(get_data_filename(animal, day, 'task'))
            filtered_epochs = [(ind, epoch) for ind, epoch in enumerate(task_file['task'][0, day-1][0])
                               if epoch['type'] == epoch_type or
                               epoch_type == '']
            epoch_index += [(day, ind) for ind, epoch in filtered_epochs
                    if ('environment' in epoch.dtype.names and
                    epoch['environment'] == environment) or
                    environment == '']
        except IOError as err:
            print('I/O error({0}): {1}'.format(err.errno, err.strerror))
            sys.exit()

    return epoch_index


def get_var(animal, day, file_type, variable, epoch_type='', environment=''):
    '''
    Returns a filtered list containing the data structures corresponding to the 
    animal, day, file_type, epoch specified.
    '''
    epoch = get_epoch(animal, day, epoch_type=epoch_type, environment=environment)
    file = scipy.io.loadmat(get_data_filename(animal, day, file_type))
    return [file[variable][0, day - 1][0, ind] for day, ind in epoch]


def get_DIO_var(animal, day, dio_var, epoch_type='', environment=''):
    '''
    Returns a list of lists given a DIO variable (pulsetimes, timesincelast, pulselength, and pulseind)
    with a length corresponding to the number of epochs (first level) and the number of active pins
    (second level)
    '''
    epoch_pins = get_var(animal, day, 'DIO', 'DIO', epoch_type=epoch_type, environment=environment)
    return [pin[0][dio_var]
            for pins in epoch_pins 
            for pin in pins.swapaxes(0,1) 
            if pin[0].dtype.names != None]


def get_pos_var(animal, day, pos_var, epoch_type='', environment=''):
    '''
    Returns a list of lists given a pos variable (time, x, y, dir, vel, x-sm, y-sm, dir-sm, and vel-sm)
    with a length corresponding to the number of epochs (first level)
    '''
    field_names = ['time', 'x', 'y', 'dir', 'vel', 'x-sm', 'y-sm', 'dir-sm', 'vel-sm']
    field_ind = [field_names.index(var) for var in pos_var]
    epoch_pos = get_var(animal, day, 'pos', 'pos', epoch_type=epoch_type, environment=environment)
    return [pos['data'][0, 0][:, field_ind]
           for pos in epoch_pos]

print(get_epoch(HPa, [1,2], epoch_type='run'))
# get_epoch(HPa, day, epoch_type='run', environment='wtr1')
# get_epoch(HPa, day, epoch_type='')
# get_epoch(HPa, day, epoch_type='sleep')
# get_epoch(HPa, day, epoch_type='sleep', environment='presleep')
# get_epoch(HPa, day, epoch_type='sleep', environment='wtr1')

[(1, 1), (1, 3), (1, 5), (2, 1), (2, 3)]


Let's look at one animal (*HPa*). The animal is running on the first w-track environment (*wtr1*).

In [729]:
HPa = Animal(directory='HPa_direct', short_name='HPa')
epoch_type = 'run'
environment = 'wtr1'
numDays = 8

plt.figure(figsize=(12, 9))

for day in range(1, numDays + 1):
    x = get_pos_var(HPa, day, 'x', epoch_type=epoch_type, environment=environment)
    y = get_pos_var(HPa, day, 'y', epoch_type=epoch_type, environment=environment)

    ax = plt.subplot(4, 4, day)
    plt.plot(x[0], y[0])
    plt.title('Day {day}'.format(day=day))
    plt.xlim([0, 150])
    plt.ylim([0, 150])
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    
plt.tight_layout()
plt.subplots_adjust(top=0.90)
plt.suptitle("Rat '{animal}' position by day".format(animal=HPa.short_name), fontsize=24)
sns.set_style("dark")

TypeError: 'int' object is not iterable

Now let's match the pulse times from the DIO inputs to the rat's position. The positions should correspond to the ends of the three arms -- the position of the food wells.

In [721]:
day = 1
print(day)
print(epoch_type)
print(environment)
print(get_epoch(HPa, day, epoch_type=epoch_type, environment=''))
# pulse_times = get_DIO_var(HPa, day, 'pulsetimes', epoch_type=epoch_type, environment=environment)
# time = get_pos_var(HPa, day, ['time', 'x', 'y'], epoch_type=epoch_type, environment=environment)
#     for epochs in pulse_times:
#         for pin in epochs:
#             for times in pin[0]:
#                 print(min(abs((times[0] / 1E5) - time)))

1
run
wtr1
[(1, 1), (1, 3), (1, 5)]
